In [1]:
print ("Hello")

Hello


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up ChromeDriver service
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Open the website
    driver.get("https://www.nse.co.ke/dataservices/market-statistics/")

    wait = WebDriverWait(driver, 60)

    # Locate and click the "Equity Statistics" tab
    equities_tab = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//a[span[text()="Equity Statistics"]]')
    ))
    equities_tab.click()

    # Wait for the table to load
    table = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "table.table.nsetable")
    ))

    # Open CSV file for writing
    with open("Shares.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Company", "ISIN Code", "Volume", "Last Traded Price", "Change"])

        # Wait for table body rows to appear
        rows = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "table.table.nsetable tbody tr")
        ))

        # Loop through rows and extract data
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 5:
                company = cells[0].text.strip()
                isin_code = cells[1].text.strip()
                volume = cells[2].text.strip()
                last_traded_price = cells[3].text.strip()
                change = cells[4].text.strip()

                writer.writerow([company, isin_code, volume, last_traded_price, change])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up ChromeDriver service
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Open the website
    driver.get("https://www.nse.co.ke/dataservices/market-statistics/")

    wait = WebDriverWait(driver, 60)

    # Locate and click the "Equity Statistics" tab
    equities_tab = wait.until(EC.element_to_be_clickable(
        (By.XPATH, '//a[span[text()="Equity Statistics"]]')
    ))
    equities_tab.click()

    # Wait for the table to load
    table = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "table.table.nsetable")
    ))

    # Wait for at least one row to have non-empty first cell (Company name)
    wait.until(lambda d: any(
        cell.text.strip() != "" for cell in d.find_elements(By.CSS_SELECTOR, "table.table.nsetable tbody tr td")
    ))

    # Open CSV file for writing
    with open("Share.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Company", "ISIN Code", "Volume", "Last Traded Price", "Change"])

        # Now fetch rows again after text has loaded
        rows = driver.find_elements(By.CSS_SELECTOR, "table.table.nsetable tbody tr")

        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 5:
                company = cells[0].text.strip()
                isin_code = cells[1].text.strip()
                volume = cells[2].text.strip()
                last_traded_price = cells[3].text.strip()
                change = cells[4].text.strip()

                print(company, isin_code, volume, last_traded_price, change)  # Debug preview
                writer.writerow([company, isin_code, volume, last_traded_price, change])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()


Kakuzi Ord.5.00 KE0000000281 12 388.5 0.00
Sasini Ltd Ord 1.00 KE0000000430 11336 18.1 -2.69
Eaagads Ltd Ord 1.25 KE0000000208 555 20.35 6.82
Williamson Tea Kenya Ltd Ord 5.00 KE0000000505 92 240 0.00
Kapchorua Tea Co. Ltd Ord Ord 5.00 KE4000001760 750 327 0.00
Limuru Tea Co. Ltd Ord 20.00 KE0000000356 0 310 0.00
    
    
    
    
    
    
    
    
    
    
